In [2]:
import copy
import os
import random
import sys

In [3]:
import numpy as np
from tqdm import trange

In [4]:
import time

In [5]:
from build_tree import (build_center_single, build_distribute_four,
                        build_distribute_nine,
                        build_in_center_single_out_center_single,
                        build_in_distribute_four_out_center_single,
                        build_left_center_single_right_center_single,
                        build_up_center_single_down_center_single)
from sampling import sample_attr, sample_attr_avail, sample_rules

In [6]:
from rendering import render_panel

In [23]:
import cv2

In [30]:
img = render_panel(row_1_3)

In [31]:
# turn on img with new window
cv2.imshow('result', img)
cv2.waitKey(1)

-1

In [32]:
# turn off all windows
cv2.destroyAllWindows()

In [7]:
class args():
    pass

In [8]:
args.num_samples = 5

In [9]:
all_configs = {"center_single": build_center_single(),
                   "distribute_four": build_distribute_four(),
                   "distribute_nine": build_distribute_nine(),
                   "left_center_single_right_center_single": build_left_center_single_right_center_single(),
                   "up_center_single_down_center_single": build_up_center_single_down_center_single(),
                   "in_center_single_out_center_single": build_in_center_single_out_center_single(),
                   "in_distribute_four_out_center_single": build_in_distribute_four_out_center_single()}

In [10]:
# for key in all_configs.keys():
key = "center_single"

In [11]:
for k in trange(args.num_samples):
    cont_num = k % 10

100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 5011.12it/s]


In [12]:
root = all_configs[key]

In [13]:
while True:
    rule_groups = sample_rules()
    new_root = root.prune(rule_groups)
    if new_root is not None:
        break

In [14]:
start_node = new_root.sample()

In [19]:
row_1_1 = copy.deepcopy(start_node)
for l in range(len(rule_groups)):
    rule_group = rule_groups[l]
    rule_num_pos = rule_group[0]
    row_1_2 = rule_num_pos.apply_rule(row_1_1)
    row_1_3 = rule_num_pos.apply_rule(row_1_2)
    for i in range(1, len(rule_group)):
        rule = rule_group[i]
        row_1_2 = rule.apply_rule(row_1_1, row_1_2)
    for i in range(1, len(rule_group)):
        rule = rule_group[i]
        row_1_3 = rule.apply_rule(row_1_2, row_1_3)
    if l == 0:
        to_merge = [row_1_1, row_1_2, row_1_3]
    else:
        merge_component(to_merge[1], row_1_2, l)
        merge_component(to_merge[2], row_1_3, l)

In [24]:
img = render_panel(row_1_1)

In [43]:
rule = rule_groups[0][3]
rule

In [44]:
dummy = rule.apply_rule(row_1_1, row_1_2)

In [45]:
rule.attr

'Color'

In [46]:
rule.component_idx

0

In [48]:
aot = row_1_1
in_aot = row_1_2

In [49]:
current_layout = aot.children[0].children[rule.component_idx].children[0]
second_aot = copy.deepcopy(in_aot)
second_layout = second_aot.children[0].children[rule.component_idx].children[0]

In [52]:
old_value_level = current_layout.children[0].type.get_value_level()
old_value_level

5

In [57]:
rule.first_col

False

In [61]:
rule.value

-1

In [58]:
current_layout.uniformity.get_value()

False

In [59]:
current_layout.children

[Entity.0]

In [55]:
# enforce value consistency
if rule.first_col and not current_layout.uniformity.get_value():
    for entity in current_layout.children:
        entity.type.set_value_level(old_value_level)
for entity in second_layout.children:
    entity.type.set_value_level(old_value_level + rule.value)

In [31]:
a = new_root.sample()

In [32]:
ent_a = a.children[0].children[0].children[0].children[0]

In [99]:
from rendering import render_entity

In [100]:
ent_a

Entity.0

In [101]:
img = render_entity(ent_a)

In [102]:
np.sum(rend_a)

3453477

In [34]:
render_panel(new_root)

AssertionError: 